# M5 data generation for next task

In [1]:
# Package imports
import numpy as np
import pandas as pd

n_lags = 14 #number of lags or lookback window of the timeseries to use for prediction
"""Please feel free to experiment with different values of n_lags"""

#Function definitions
def reshape_timeseries(y_series, n_lags):
    """
    This function reshapes a timeseries into input-output pairs {Xtrain,ytrain}
    y_series: input timeseries
    n_lags: number of lags or lookback window of the timeseries
    """
    X_train = []
    y_train = []
    for i in range(n_lags, len(y_series)):
        ar = y_series[i-n_lags: i]
        X_train.append(ar)
        y_train.append(y_series[i])
    return np.array(X_train).reshape(-1,n_lags), np.array(y_train).reshape(-1,1)

#Reading csv
level12_data = pd.read_csv('sales_train_evaluation.csv', low_memory = False) #original dataset is at level 12 aggregation

#Aggregation: group data for each of 10 different stores
level3_data = level12_data.groupby('store_id').sum() #after grouping by store_id, we obtain data at level 3 aggregation

#Dictionary comprehension to build input-output pairs in the form {X,y} from timeseries data for all stores
data_dict = {store_id: reshape_timeseries(list(level3_data.loc[store_id,:].values), n_lags) for store_id in level3_data.index}

## _Example usage_

There are $10$ different unique store_ids in the dataset as follows:

In [2]:
level3_data.index

Index(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2',
       'WI_3'],
      dtype='object', name='store_id')

The reshaped data for all 10 stores are stored in the dictionary _data_dict_.

If we are interested in the reshaped data for store 'CA_2', we use:

In [3]:
X, y = data_dict['CA_2']

If we are interested in the data for 'TX_3', we use:

In [4]:
X, y = data_dict['TX_3']

Note that {$X,y$} contain both training and validation examples. To get training and validation splits, you may use below:

In [5]:
X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
# Validation is to be done on the last 28 days of data

In [6]:
X_train.shape, y_train.shape

((1899, 14), (1899, 1))

In [7]:
X_val.shape, y_val.shape

((28, 14), (28, 1))

## Solution

In [8]:
from lightgbm import LGBMRegressor as lgb
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE, r2_score as R2

In [9]:
results = []
for store_id in level3_data.index:
    
    #Getting store X,y data
    X, y = data_dict[store_id]  
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    # Validation is to be done on the last 28 days of data
    
    #Fitting gradient boosted tree
    lgbm = lgb(random_state=42).fit(X_train, y_train.flatten())
    y_pred = lgbm.predict(X_val)
    
    # Error metrics
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred)
    results.append([r2, rmse, mae])
    
results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'R2', 1: 'RMSE', 2: 'MAE'})
results_df

,R2,RMSE,MAE
CA_1,0.820680,385.671730,304.971705
CA_2,0.786946,525.425365,426.964316
CA_3,0.795791,399.007473,310.060724
CA_4,0.372608,252.844423,199.242327
TX_1,0.239947,504.211696,357.129869
TX_2,0.412687,450.819057,321.472026
TX_3,0.296915,429.678230,342.235209
WI_1,0.615731,493.399962,383.517901
WI_2,0.259336,855.080991,634.295937
WI_3,0.579789,495.591349,400.397303


### Root mean squared scaled error

In [10]:
denom = np.mean(np.diff(y_train.flatten())**2)
rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
rmsse

0.6982853341723568

$(y_t - y_{t-1})^2$

In [11]:
results = []
for store_id in level3_data.index:
    #Getting store X,y data
    X, y = data_dict[store_id]  
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    # Validation is to be done on the last 28 days of data
    
    #RMSSE denominator
    ytrain = level3_data.loc[store_id,:].values[0:-28]
    denom = np.mean(np.diff(ytrain.flatten())**2)
    
    #Fitting gradient boosted tree
    lgbm = lgb(random_state=42).fit(X_train, y_train.flatten())
    y_pred = lgbm.predict(X_val)
    
    # Error metrics
    rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred)
    
    results.append([rmsse, rmse, r2, mae])
    
results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'RMSSE', 1: 'RMSE', 2: 'R2', 3:'MAE'})
print('mean results:', np.mean(results_df.values, axis=0))
results_df

mean results: [  0.71116301 479.1730277    0.51804304 368.02873171]


,RMSSE,RMSE,R2,MAE
CA_1,0.445352,385.671730,0.820680,304.971705
CA_2,0.693159,525.425365,0.786946,426.964316
CA_3,0.414798,399.007473,0.795791,310.060724
CA_4,0.798875,252.844423,0.372608,199.242327
TX_1,0.844677,504.211696,0.239947,357.129869
TX_2,0.591245,450.819057,0.412687,321.472026
TX_3,0.752239,429.678230,0.296915,342.235209
WI_1,0.688044,493.399962,0.615731,383.517901
WI_2,1.188165,855.080991,0.259336,634.295937
WI_3,0.695075,495.591349,0.579789,400.397303


### Hyperparameter search

In [12]:
from sklearn.model_selection import GridSearchCV
#Alternatively use Bayesian optimisation (hyperopt) if grid space is too large

In [13]:
grid = GridSearchCV(lgb(random_state=42), 
param_grid={\
'learning_rate': [0.001, 0.01, 0.1], \
'num_leaves': [31, 40, 50], \
#'boosting_type': ['gbdt','dart'], 
#'max_depth': [-1, 4], \
#'subsample':[0.5,0.8,1], \
'colsample_bytree':[0.5,0.8,1], \
#'reg_alpha': [0,0.1], \
#'reg_lambda': [0,0.1]
}, \
scoring='neg_median_absolute_error')

In [14]:
results = []
for store_id in level3_data.index:
    print(store_id)
    #Getting store X,y data
    X, y = data_dict[store_id]  
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    # Validation is to be done on the last 28 days of data
    
    #RMSSE denominator
    ytrain = level3_data.loc[store_id,:].values[0:-28]
    denom = np.mean(np.diff(ytrain.flatten())**2)
    
    #Fitting gradient boosted tree
    grid = GridSearchCV(lgb(random_state=42), 
    param_grid={\
    'learning_rate': [0.001, 0.01, 0.1], \
    'num_leaves': [31, 40, 50], \
    #'boosting_type': ['gbdt','dart'], 
    #'max_depth': [-1, 4], \
    'subsample':[0.5,0.8,1], \
    #'colsample_bytree':[0.5,0.8,1], \
    #'reg_alpha': [0,0.1], \
    #'reg_lambda': [0,0.1]
    }, \
    scoring='neg_mean_absolute_error')
    grid.fit(X_train, y_train.flatten())
    lgbm = grid.best_estimator_
    y_pred = lgbm.predict(X_val)
    
    # Error metrics
    # Error metrics
    rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred)
    
    results.append([rmsse, rmse, r2, mae])
    
results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'RMSSE', 1: 'RMSE', 2: 'R2', 3:'MAE'})


results_df

CA_1
CA_2
CA_3
CA_4
TX_1
TX_2
TX_3
WI_1
WI_2
WI_3


,RMSSE,RMSE,R2,MAE
CA_1,0.445352,385.671730,0.820680,304.971705
CA_2,0.693159,525.425365,0.786946,426.964316
CA_3,0.435326,418.753933,0.775079,323.570944
CA_4,0.798875,252.844423,0.372608,199.242327
TX_1,0.844677,504.211696,0.239947,357.129869
TX_2,0.597484,455.576428,0.400226,313.371144
TX_3,0.797469,455.513265,0.209825,361.860591
WI_1,0.688044,493.399962,0.615731,383.517901
WI_2,1.175351,845.858998,0.275226,617.451092
WI_3,0.695075,495.591349,0.579789,400.397303


### Stationarise timeseries

In [15]:
original series: 10, 100, 120, 140, 160, 200, 210, 240

differenced: 90, 20, 20, 20, 40, 10, 30
    
    -70, 0, 0, 20, -30, 20

SyntaxError: invalid syntax (<ipython-input-15-5ba8527091f5>, line 1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(level3_data.loc['WI_3',:].values)
plt.show()

In [ ]:
#Aggregation: group data for each of 10 different stores
level3_data = level12_data.groupby('store_id').sum() #after grouping by store_id, we obtain data at level 3 aggregation

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(level3_data.loc['CA_1',:].values)

In [ ]:
def difference(original_series): 
    """
    This function tests for stationarity on the timeseries using the augmented Dickey-Fuller test.
    Non-stationary timeseries are differenced incrementally, and the stationarity test performed.
    
    original_series: input original timeseries data
    stationary_series: output stationary timeseries
    first_values: first values of the timeseries required to recover the original series; 
    the length of first_values is equal to the differencing order
    """
    first_values = []; stationary_series = original_series
    for i in range(0, 11):
        if i == 0:
            series_to_test = original_series
        else:
            series_to_test = series_to_test.diff(1)[1:]
        first_values.append(series_to_test.iloc[0])
        stat_result = adfuller(series_to_test) #Check stationarity
        pval = stat_result[1] #p-value
        if pval <= 0.05:
            stationary_series = series_to_test
            break       
    first_values = np.flip(first_values[0:-1])
    return stationary_series, first_values

In [ ]:
def integrate(differenced_series, first_values): 
    """
    This function reverse-differences a differenced timeseries to recover the original timseries
    
    differenced_series: input differenced timeseries
    original_series: output original timeseries
    """
    diff_order = len(first_values)
    original_series = differenced_series.flatten()
    if diff_order == 0:
        original_series = differenced_series.flatten()
    else:
        for i in range(diff_order):
            original_series = np.column_stack((first_values[i].reshape(1,1), (np.cumsum(original_series) \
              + first_values[i]).reshape(1,-1))).flatten()      
    return original_series

In [ ]:
results = []
for store_id in level3_data.index:
    print(store_id)
    
    # Stationarise timeseries
    stationary_series, first_values = difference(level3_data.loc[store_id,:])
    print('differencing order', len(first_values))
    
    # Reshaping timeseries into X,y pairs
    X, y = reshape_timeseries(list(stationary_series), n_lags)
    X_train, y_train, X_val, _ = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    y_val = level3_data.loc[store_id,:].values[-28:]
     
    #RMSSE denominator
    ytrain = level3_data.loc[store_id,:].values[0:-28]
    denom = np.mean(np.diff(ytrain.flatten())**2)
    
    #Fitting gradient boosted tree
    lgbm = lgb(random_state=42).fit(X_train, y_train.flatten())
    y_hat = lgbm.predict(X_val)
    
    # Recover original series (integrate)
    y_pred = level3_data.loc[store_id,:].values[-29:-1] + y_hat
    
    # Error metrics
    rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred)
    
    results.append([rmsse, rmse, r2, mae])

results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'RMSSE', 1: 'RMSE', 2: 'R2', 3:'MAE'})
print('mean results:', np.mean(results_df.values, axis=0))
results_df

#### Stationarised timeseries example (WI_3)

In [ ]:
plt.plot(stationary_series.values)

### Learn across all series

In [ ]:
CA1: 
CA2: 

In [ ]:
Xtrain = pd.DataFrame()
ytrain = pd.DataFrame()
Xval = pd.DataFrame()
yval = pd.DataFrame()
store_count = 0
for store_id in level3_data.index:
    X, y = data_dict[store_id]
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    
    if store_id.split('_')[0] == 'CA':
        state_id = 1
    elif store_id.split('_')[0] == 'TX':
        state_id = 2
    elif store_id.split('_')[0] == 'WI':
        state_id = 3
    
    X_train = np.column_stack((store_count*np.ones([X_train.shape[0],1],dtype='int64'), state_id*np.ones([X_train.shape[0],1],dtype='int64'), X_train))
    X_val = np.column_stack((store_count*np.ones([X_val.shape[0],1],dtype='int64'), state_id*np.ones([X_val.shape[0],1],dtype='int64'), X_val))
    
    Xtrain = pd.concat([Xtrain, pd.DataFrame(X_train)])
    ytrain = pd.concat([ytrain, pd.DataFrame(y_train)])
    
    Xval = pd.concat([Xval, pd.DataFrame(X_val)])
    yval = pd.concat([yval, pd.DataFrame(y_val)])
    
    store_count = store_count + 1

In [ ]:
Xtrain

In [ ]:
exog_train = pd.get_dummies(Xtrain.iloc[:,[0,1]], columns=[0,1], drop_first=True)
Xtrain = pd.concat([exog_train, Xtrain], axis=1, join='inner')
Xtrain

In [ ]:
ytrain

In [ ]:
exog_val = pd.get_dummies(Xval.iloc[:,[0,1]], columns=[0,1], drop_first=True)
Xval = pd.concat([exog_val,Xval], axis=1, join='inner')
Xval

In [ ]:
yval

In [ ]:
#Fitting gradient boosted tree
lgbm = lgb(random_state=42).fit(Xtrain, ytrain)
yhat = lgbm.predict(Xval)
YHAT = yhat.reshape(10,28)

In [ ]:
# Error metrics
results = []
for i in range(10):
    #RMSSE denominator
    store_id = level3_data.index[i]
    X, y = data_dict[store_id]
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    
    ytrain = level3_data.loc[store_id,:].values[0:-28]
    denom = np.mean(np.diff(ytrain.flatten())**2)
    
    y_pred = YHAT[i,:]
    # Error metrics
    rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred) 
    results.append([rmsse, rmse, r2, mae])
    
results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'RMSSE', 1: 'RMSE', 2: 'R2', 3:'MAE'})
print('mean results:', np.mean(results_df.values, axis=0))
results_df

### Combine all

In [ ]:
Xtrain = pd.DataFrame()
ytrain = pd.DataFrame()
Xval = pd.DataFrame()
yval = pd.DataFrame()
store_count = 0
for store_id in level3_data.index:
    
     # Stationarise timeseries
    stationary_series, first_values = difference(level3_data.loc[store_id,:])
    print('differencing order', len(first_values))
    
    # Reshaping timeseries into X,y pairs
    X, y = reshape_timeseries(list(stationary_series), n_lags)
    X_train, y_train, X_val, _ = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    y_val = level3_data.loc[store_id,:].values[-28:]
    
    
    if store_id.split('_')[0] == 'CA':
        state_id = 1
    elif store_id.split('_')[0] == 'TX':
        state_id = 2
    elif store_id.split('_')[0] == 'WI':
        state_id = 3
    
    X_train = np.column_stack((store_count*np.ones([X_train.shape[0],1],dtype='int64'), state_id*np.ones([X_train.shape[0],1],dtype='int64'), X_train))
    X_val = np.column_stack((store_count*np.ones([X_val.shape[0],1],dtype='int64'), state_id*np.ones([X_val.shape[0],1],dtype='int64'), X_val))
    
    Xtrain = pd.concat([Xtrain, pd.DataFrame(X_train)])
    ytrain = pd.concat([ytrain, pd.DataFrame(y_train)])
    
    Xval = pd.concat([Xval, pd.DataFrame(X_val)])
    yval = pd.concat([yval, pd.DataFrame(y_val)])
    
    store_count = store_count + 1

In [ ]:
#Fitting gradient boosted tree
grid = GridSearchCV(lgb(random_state=42), 
param_grid={\
'learning_rate': [0.001, 0.01, 0.1], \
'num_leaves': [31, 40, 50], \
#'boosting_type': ['gbdt','dart'], 
#'max_depth': [-1, 4], \
'subsample':[0.5,0.8,1], \
#'colsample_bytree':[0.5,0.8,1], \
#'reg_alpha': [0,0.1], \
#'reg_lambda': [0,0.1]
}, \
scoring='neg_mean_absolute_error')
grid.fit(Xtrain, ytrain)
lgbm = grid.best_estimator_
yhat = lgbm.predict(Xval)
YHAT = yhat.reshape(10,28)
YVAL = yval.values.reshape(10,28)

In [ ]:
# Error metrics
results = []
for i in range(10):
    #RMSSE denominator
    store_id = level3_data.index[i]
    X, y = data_dict[store_id]
    
    ytrain = level3_data.loc[store_id,:].values[0:-28]
    denom = np.mean(np.diff(ytrain.flatten())**2)
    
    y_pred = YHAT[i,:] + level3_data.loc[store_id,:].values[-29:-1]
    y_val = YVAL[i,:]
    # Error metrics
    rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred)
    results.append([rmsse, rmse, r2, mae])
    
results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'RMSSE', 1: 'RMSE', 2: 'R2', 3:'MAE'})
print('mean results:', np.mean(results_df.values, axis=0))
results_df

### Multistep prediction (Next steps)

In [ ]:
y = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3 /45, 101, 0, 7, 8

1,2,3 -- 4
2,3,4 -- 5
...

0,1,2 -- 3

test example:
1,2,3 -- x= 44/ 45

2,3,44 -- x= 95/ 101
3,44,95 -- x = -12/ 0
44,95,-12 -- x=23 /7


45,101,0 -- 7/ 6.5
101,0,7 -- 8/ 8.2

## RBF-DiffNet

In [ ]:
# Here, I'm trying out a network I developed on the M5 problem

In [ ]:
import time
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy.special import binom, factorial
from sklearn.cluster import KMeans
from scipy.optimize import minimize
from scipy.spatial import KDTree

from statsmodels.tsa.stattools import adfuller
import pmdarima as pm

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as MSE
from sklearn.linear_model import LassoCV, LinearRegression as LR

#Set random seed
random.seed(42)
np.random.seed(42)

In [ ]:
def variableBetas(X, c):
    """
    The function returns c RBF centres and widths using K-Means clustering, with each RBF having its own unique width.
    
    X: input matrix of size n-by-d, where n is the number of training examples, and d is the number of features
    c: the number of RBF centres
    centres: output of size c-by-d
    betas: output of size 1-by-d
    """
    clusterObj = KMeans(n_clusters = c, random_state = 42).fit(X)
    labels = clusterObj.labels_
    p = 10
    betas = []
    centres = []
    for label in np.unique(labels):
        Xk = X[labels == label, :]
        Muk = np.mean(Xk, axis=0)
        tree = KDTree(Xk)
        dref, iref = tree.query(np.mean(Xk, axis=0).reshape(1,-1), p)
        #Muk = Xk[iref[0],:]
        #sigma = np.mean(np.linalg.norm(Xk - Muk, axis = 1), axis = 0)
        sigma = np.mean(dref)
        betas.append(1/(2*sigma**2))
        centres.append(Muk)      
    centres = np.array(centres)
    betas = np.array(betas); betas[np.isinf(betas)] = np.sum(betas[np.isinf(betas)==False])
    return centres, betas

def rbf(X, centres, betas):
    """
    This function evaluates the radial basis function (RBF) Phi for each data sample in X.
    
    X: input matrix of size N-by-d, where N is the number of training examples, and d is the number of features
    centres: input array of RBF locations of size: c-by-dy, where c is the number of centres
    betas: input array of RBF widths of size: 1-by-d
    Phi: an output array of size N-by-c
    """
    N, d = X.shape 
    Phi = [np.exp(-betas* np.linalg.norm(X[n] - centres, axis = 1)**2) for n in range(N)]  
    return np.array(Phi)

def trainRBFN(Xtrain, ytrain, c, normalise): 
    """
    This function trains the weights for the normalised and unnormalised RBF networks
    
    Xtrain: training input of size n_train-by-d
    ytrain: training output of size n_train-by-1
    c: number of radial basis function (RBF) centres
    normalise: a Boolean input indicating normalised RBF network (True) or unnormalised RBF network (False)
    outputs: linear model at output layer, together with the RBF centres and widths 
    """
    n, d = Xtrain.shape 
    centres, betas = variableBetas(Xtrain, c)
    Phi = rbf(Xtrain, centres, betas)    
    if normalise == True:
        Phi = Phi/ np.sum(Phi, axis = 1).reshape(-1,1)
    linMdl = LassoCV().fit(Phi, ytrain)
    mse = MSE(linMdl.predict(Phi), ytrain)
    return linMdl, centres, betas

def testRBFN(Xtest, centres, betas, linMdl, normalise):
    """
    This function tests the normalised/unnormalised RBF network on new samples
    
    Xtest: test input of size n_test-by-d
    centres: input array of RBF locations of size: c-by-d, where c is the number of centres
    betas: input array of RBF widths of size: 1-by-d
    linMdl: linear model at output layer of RBF network
    normalise: a Boolean input indicating normalised RBF network (True) or unnormalised RBF network (False)
    outputs: one step predictions of size: n_test-by-1 - for the sequence 
    """
    n, d = Xtest.shape
    Phi = rbf(Xtest, centres, betas)
    if normalise == True:
        Phi = Phi/ np.sum(Phi, axis = 1).reshape(-1,1)    
    ypred = linMdl.predict(Phi)  
    return ypred   

def rbfderivatives(X, centres, betas, order, Phi):
    """
    This function evaluates the radial basis function (RBF) derivatives with respect to each component of x 
    up to a differential order of "order", where x is given by each row of X.
    
    X: input matrix of size N-by-d, where N is the number of training examples, and d is the number of features
    centres: input array of RBF locations of size: c-by-dy, where c is the number of centres
    betas: input array of RBF widths of size: 1-by-d
    Phi: input array of radial basis functions of size N-by-c
    order: input scalar representing the order of the differential equation
    diPhi: output array of RBF derivatives of size: N-by-order-by-d-by-c
    """
    N, d = X.shape
    c = centres.shape[0]
    diPhi = []
    for n in range(N): 
        diPhi_n = []
        diPhi_n.append(np.tile(Phi[n], (d, 1))) 
        for i in range(1, order+1):          
            leibniz_sum = 0    
            for k in range(i):       
                bin_coeff = binom(i-1,k)               
                if i-k-1 == 0:
                    u = (-2*betas.reshape(-1,1)*(X[n]-centres)).T
                elif i-k-1 == 1:
                    u = (-2*betas.reshape(-1,1)* np.ones([c, d])).T
                elif i-k-1 >= 2:
                    u = np.zeros([d, c])             
                leibniz_sum = leibniz_sum + (bin_coeff* u* diPhi_n[k])         
            diPhi_n.append(leibniz_sum)         
        diPhi.append(diPhi_n[1:]) #Excluding the zeroth derivative
    return np.array(diPhi)

def obj_func(x, extraArgs):
    """
    The function evaluates the loss function for training the differential RBF network weights.
    
    x: initial solution of weights to optimise
    extraArgs: extra arguments to compute loss function
    output: loss function value
    """
    diPhi = extraArgs[0]
    y = extraArgs[1]
    ylagged = extraArgs[2]
    n, _, c = diPhi.shape
    nlags = ylagged.shape[1]  
    w_lagged = x[0:nlags].reshape(-1,1)
    rbf_coeffs = x[nlags:nlags+c].reshape(-1,1)    
    pde_coeffs = x[nlags+c:].reshape(-1,1)
    ypred = ((np.sum(pde_coeffs.reshape(1, len(pde_coeffs),1)*diPhi, axis=1))@rbf_coeffs) + ylagged@w_lagged
    return MSE(y, ypred)

def trainRBFDiffNet(Xtrain, ytrain, c, nlags, order): #This function trains the proposed RBF network
    """
    This function trains the differential RBF network (RBF-DiffNet)
    
    Xtrain: training input of size n_train-by-d
    ytrain: training output of size n_train-by-1
    c: number of radial basis function (RBF) centres
    nlags: number of lags or lookback window of the timeseries
    order: order of the partial differential equation
    outputs: network weights together with the RBF centres and widths
    """
    n_train, d = Xtrain.shape
    centres, betas = variableBetas(Xtrain, c)

    ylagged = Xtrain[:,-nlags:]
    if nlags == 1:
        ylagged = ylagged.reshape(-1,1)
    Phi = rbf(Xtrain, centres, betas)
    diPhi = rbfderivatives(Xtrain, centres, betas, order, Phi).reshape([n_train, order*d, c])

    rbfMdl = LR(fit_intercept=True).fit(Phi, ytrain)
    #rbfMdl = LassoCV(fit_intercept=True).fit(Phi, ytrain)
    rbf_coeffs = rbfMdl.coef_.reshape(-1,1)
    bias = rbfMdl.intercept_
    w_lagged = np.ones([nlags,1])/nlags
    #pde_coeffs = np.zeros([order*d,1])
    pde_coeffs = (np.ones([order, d])*(0.1**np.arange(1,order+1)/factorial(np.arange(1,order+1))).reshape(-1,1)).reshape(-1,1)
    ytrain = ytrain.reshape(-1,1)
    
    ypred = ((np.sum(pde_coeffs.reshape(1, len(pde_coeffs),1)*diPhi, axis=1))@rbf_coeffs) + ylagged@w_lagged
    min_err = MSE(ytrain, ypred)
    opt_weights = [rbf_coeffs, pde_coeffs, w_lagged, bias]
    
    num_iter = 100
    for i in range(num_iter):
        #print(i)
        #Step 1: Fix w_lagged, rbf_coeffs; solve for pde_coeffs
        pde_coeffs = np.linalg.pinv((np.sum(rbf_coeffs.reshape(1,1,c)*diPhi, axis=2)))@(ytrain - ylagged@w_lagged)

        #Step 2: Fix rbf_coeffs, pde_coeffs; solve for w_lagged
        w_lagged = np.linalg.pinv(ylagged)@(ytrain - ((np.sum(pde_coeffs.reshape(1, len(pde_coeffs),1)*diPhi, axis=1))@rbf_coeffs))

        #Step 3: Fix w_lagged, pde_coeffs; solve for rbf_coeffs
        rbf_coeffs = np.linalg.pinv((np.sum(pde_coeffs.reshape(1, len(pde_coeffs),1)*diPhi, axis=1)))@(ytrain - ylagged@w_lagged)
         
        bias = np.mean(ytrain - Phi@rbf_coeffs)
        
        #Compute predictions and errors
        ypred = ((np.sum(pde_coeffs.reshape(1, len(pde_coeffs),1)*diPhi, axis=1))@rbf_coeffs) + ylagged@w_lagged
        err = MSE(ytrain, ypred)
        
        if err < min_err:
            min_err = err
            opt_weights = [rbf_coeffs, pde_coeffs, w_lagged, bias]

    return opt_weights, centres, betas

def testRBFDiffNet(Xtest, weights, centres, betas):
    """
    This function tests the differential RBF network on new samples
    
    Xtest: test input of size n_test-by-d
    weights: differential RBF network weights
    centres: input array of RBF locations of size: c-by-d, where c is the number of centres
    betas: input array of RBF widths of size: 1-by-d
    order: order of the partial differential equation
    outputs: one step predictions of size: n_test-by-1 - for the sequence 
    """
    [rbf_coeffs, pde_coeffs, w_lagged, bias] = weights
    c = len(betas)
    n_test, d = Xtest.shape
    order = int(len(pde_coeffs)/d)
    nlags = len(w_lagged)
    ylagged = Xtest[:,-nlags:]  
    Phi = rbf(Xtest, centres, betas)   
    diPhi = rbfderivatives(Xtest, centres, betas, order, Phi).reshape([n_test, order*d, c])
    ypred = ((np.sum(pde_coeffs.reshape(1, len(pde_coeffs),1)*diPhi, axis=1))@rbf_coeffs) + ylagged@w_lagged
    #ypred = Phi@rbf_coeffs + bias
    return ypred


def one_step_predict(algorithm):
    """
    This function provides one-step predictions for a timeseries

    algorithm: an input string indicating what algorithm is being used for the predictions
    yhat: onestep predictions
    """
    ytest_scaled = scaler.transform(ytest.reshape(-1,1))
    ylist = list(ytrain_scaled.flatten()) + list(ytest_scaled.flatten())
    ylagged, ynext = reshape_timeseries(ylist, nlags)
    X_test = ylagged[-num_test:, :].reshape(num_test, -1)

    if algorithm == 'proposed':
        ypred = testRBFDiffNet(X_test, weights, centres, betas)
    elif algorithm == 'urbfn':
        ypred = testRBFN(X_test, centres_u, betas_u, rbfMdl_u, False)  
    elif algorithm == 'nrbfn':
        ypred = testRBFN(X_test, centres_n, betas_n, rbfMdl_n, True)
    yhat = scaler.inverse_transform(ypred.reshape(-1,1))
    return yhat

In [ ]:
c = 28
nlags = 14
order_opt = 1

In [ ]:
results = []
for store_id in level3_data.index:
    print('training ', store_id)
    
    #Getting store X,y data
    X, y = data_dict[store_id]  
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    # Validation is to be done on the last 28 days of data
    
    #RMSSE denominator
    ytrain = level3_data.loc[store_id,:].values[0:-28]
    denom = np.mean(np.diff(ytrain.flatten())**2)
     
    
    # Fit RBF-DiffNet
    weights, centres, betas = trainRBFDiffNet(X_train, y_train, c, nlags, order_opt)
    
    # Predict 
    y_pred = testRBFDiffNet(X_val, weights, centres, betas)
    

   # Error metrics
    rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred)
    
    results.append([rmsse, rmse, r2, mae])
    
results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'RMSSE', 1: 'RMSE', 2: 'R2', 3:'MAE'})
print('mean results:', np.mean(results_df.values, axis=0))
results_df

### Learning across the series with RBF-DiffNet

In [ ]:
Xtrain = pd.DataFrame()
ytrain = pd.DataFrame()
Xval = pd.DataFrame()
yval = pd.DataFrame()
store_count = 0
for store_id in level3_data.index:
    X, y = data_dict[store_id]
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    
    if store_id.split('_')[0] == 'CA':
        state_id = 1
    elif store_id.split('_')[0] == 'TX':
        state_id = 2
    elif store_id.split('_')[0] == 'WI':
        state_id = 3
    
    X_train = np.column_stack((store_count*np.ones([X_train.shape[0],1],dtype='int64'), state_id*np.ones([X_train.shape[0],1],dtype='int64'), X_train))
    X_val = np.column_stack((store_count*np.ones([X_val.shape[0],1],dtype='int64'), state_id*np.ones([X_val.shape[0],1],dtype='int64'), X_val))
    
    Xtrain = pd.concat([Xtrain, pd.DataFrame(X_train)])
    ytrain = pd.concat([ytrain, pd.DataFrame(y_train)])
    
    Xval = pd.concat([Xval, pd.DataFrame(X_val)])
    yval = pd.concat([yval, pd.DataFrame(y_val)])
    
    store_count = store_count + 1

In [ ]:
# Fitting RBF-DiffNet

order_opt = 1
nlags = 2
c = 50

weights, centres, betas = trainRBFDiffNet(Xtrain.values, ytrain.values, c, nlags, order_opt)
yhat = testRBFDiffNet(Xval.values, weights, centres, betas)
YHAT = yhat.reshape(10,28)

In [ ]:
# Error metrics
results = []
for i in range(10):
    #RMSSE denominator
    store_id = level3_data.index[i]
    X, y = data_dict[store_id]
    X_train, y_train, X_val, y_val = X[0:-28, :], y[0:-28, :], X[-28:, :], y[-28:, :]
    
    ytrain = level3_data.loc[store_id,:].values[0:-28]
    denom = np.mean(np.diff(ytrain.flatten())**2)
    
    y_pred = YHAT[i,:]
    # Error metrics
    rmsse = np.sqrt(MSE(y_val, y_pred)/denom)
    r2 = R2(y_val, y_pred)
    rmse = MSE(y_val, y_pred, squared=False)
    mae = MAE(y_val, y_pred) 
    results.append([rmsse, rmse, r2, mae])
    
results_df = pd.DataFrame(results)
results_df = results_df.rename(index={results_df.index[i]: level3_data.index[i] for i in range(10)}, columns={0: 'RMSSE', 1: 'RMSE', 2: 'R2', 3:'MAE'})
print('mean results:', np.mean(results_df.values, axis=0))
results_df